###### Visualización de Datos - Práctica 1

# Preparación de los datos

In [1]:
#Imports
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os

In [2]:
#Carga datasets
salaries_path = '../data/raw/salaries.csv'
cost_of_living_path = '../data/raw/cost_of_living.csv'
iso_country_codes_path = '../data/raw/iso_country_codes.csv'

df_salaries = pd.read_csv(salaries_path)
df_cost_of_living = pd.read_csv(cost_of_living_path, delimiter=';')
df_iso_country_codes = pd.read_csv(iso_country_codes_path, delimiter=';')

In [3]:
df_salaries.sample(3)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
72575,2022,SE,FT,Data Engineer,184100,USD,184100,US,0,US,M
42284,2024,SE,FT,Data Scientist,149500,USD,149500,US,0,US,M
22060,2024,MI,FT,Data Operations Engineer,90000,USD,90000,US,100,US,M


In [4]:
df_cost_of_living.sample(3)

,Rank,Country,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
103,104,Philippines,"33,6","8,3","21,6","35,2","17,9","25,9"
128,129,Paraguay,"27,6","10,5","19,5","24,9","19,1","28,1"
78,79,Mauritius,"39,8",11,"26,2","41,4","27,1","32,8"


In [5]:
df_iso_country_codes.sample(3)

,Country,Alpha-2 code,Alpha-3 code,Numeric
185,Saint Kitts and Nevis,KN,KNA,659
90,Guernsey,GG,GGY,831
20,Belgium,BE,BEL,56


Para poder combinar la información de los dataframes y pensando en mostrar los nombres enteros de los países en las visualizaciones sustituimos los códigos de las columnas "employee_residence' y 'company_location' por el nombre del país completo:

In [6]:
# Diccionario código - nombre país
iso_mapping = df_iso_country_codes.set_index('Alpha-2 code')['Country'].to_dict()

# Cambiar los códigos por los nombres
df_salaries['employee_residence'] = df_salaries['employee_residence'].map(iso_mapping)
df_salaries['company_location'] = df_salaries['company_location'].map(iso_mapping)


df_salaries.sample(10)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
3805,2024,SE,FT,Machine Learning Engineer,240000,USD,240000,United States,0,United States,M
70802,2023,MI,FT,Data Scientist,106250,USD,106250,United States,0,United States,M
19556,2024,SE,FT,Engineer,149020,USD,149020,United States,0,United States,M
73615,2020,SE,FT,Lead Data Scientist,190000,USD,190000,United States,100,United States,S
133,2025,SE,FT,Data Architect,102400,USD,102400,United States,100,United States,M
7868,2024,SE,FT,Software Developer,96800,USD,96800,United States,0,United States,M
17557,2024,SE,FT,Applied Scientist,250000,USD,250000,Canada,0,Canada,M
3847,2024,EN,FT,Analytics Engineer,79700,USD,79700,United States,0,United States,M
73678,2021,SE,FT,Data Analytics Manager,140000,USD,140000,United States,100,United States,L
67592,2023,SE,FT,Machine Learning Engineer,140000,USD,140000,United States,0,United States,M


Combinamos los datos para crear un único archivo con el que crearemos las visualizaciones:

In [7]:
df_salaries = df_salaries.merge(df_cost_of_living, left_on="employee_residence", right_on="Country", how="left")

In [8]:
df_salaries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73719 entries, 0 to 73718
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   work_year                       73719 non-null  int64  
 1   experience_level                73719 non-null  object 
 2   employment_type                 73719 non-null  object 
 3   job_title                       73719 non-null  object 
 4   salary                          73719 non-null  int64  
 5   salary_currency                 73719 non-null  object 
 6   salary_in_usd                   73719 non-null  int64  
 7   employee_residence              73719 non-null  object 
 8   remote_ratio                    73719 non-null  int64  
 9   company_location                73719 non-null  object 
 10  company_size                    73719 non-null  object 
 11  Rank                            73715 non-null  float64
 12  Country                         

In [9]:
# Eliminamos las columnas que ya no queremos
df_salaries = df_salaries.drop(columns=['Rank','Country'])

# Renombramos las que necesitamos para el análisis
df_salaries.columns = df_salaries.columns.str.lower().str.replace(' ', '_')

# Eliminamos los 4 registros con cost_of_living_index nulo, de países 
df_salaries = df_salaries[df_salaries['cost_of_living_index'].notna()]

Debemos tratar los formatos de las columnas para alinearlos con la herramienta de visualización que se utilizará. En este caso Tableau. Al tenerlo en Español, una vez hayamos reemplazado las comas por puntos para realizar los calculos de las métricas deseadas deberemos volver a reemplazarlos por comas. 

In [10]:
# Reemplazar las comas por puntos en las columnas relevantes y convertir las columnas a float
df_salaries['cost_of_living_index'] = df_salaries['cost_of_living_index'].str.replace(',', '.').astype(float)
df_salaries['rent_index'] = df_salaries['rent_index'].str.replace(',', '.').astype(float)
df_salaries['cost_of_living_plus_rent_index'] = df_salaries['cost_of_living_plus_rent_index'].str.replace(',', '.').astype(float)
df_salaries['groceries_index'] = df_salaries['groceries_index'].str.replace(',', '.').astype(float)
df_salaries['restaurant_price_index'] = df_salaries['restaurant_price_index'].str.replace(',', '.').astype(float)
df_salaries['local_purchasing_power_index'] = df_salaries['local_purchasing_power_index'].str.replace(',', '.').astype(float)

Se observó en el análisis previo que existe muy poco dato previo al 2024 por lo que filtraremos los datos a partir de este año y, además, filtraremos los paises con menos de 10 ocurrencias en el dataset ya que podrían sesgar los resultados.

In [11]:
# Filtrar por los años 2024 y 2025
df_salaries_filtered_years = df_salaries[df_salaries['work_year'].isin([2024, 2025])]

# Contar los registros por país de residencia en los años 2024 y 2025
country_counts = df_salaries_filtered_years['employee_residence'].value_counts()
country_counts.head()

NameError: name 'country_counts' is not defined

Para enriquecer el dataset y el futuro análisis se incluyen dos métricas utilizando columnas existentes:
  - salario ajustado en base al poder adquisitivo
  - salario / costo de vida 
  
Creamos un df a parte con el listado de países y estas dos columnas para usarla como dimensión en el modelo de datos.
    

In [ ]:
df_salaries['salario_costo_vida'] = df_salaries['salary_in_usd'] / df_salaries['cost_of_living_index'].astype(float)
df_salaries['salario_ajustado'] = df_salaries['salary_in_usd'] * (df_salaries['local_purchasing_power_index'].astype(float) / 100)

# Agrupar por país y calcular la media de las métricas 
df_pais_avg = df_salaries.groupby('employee_residence')[['salario_costo_vida', 'salario_ajustado']].mean().reset_index()
df_pais_avg['salario_costo_vida'] = df_pais_avg['salario_costo_vida'].round(2)
df_pais_avg['salario_ajustado'] = df_pais_avg['salario_ajustado'].round(2)


df_pais_avg.sample(5)

Volvemos a reemplazar los puntos por comas como se ha comentado anteriormente para ajustarse a los parámetros de Tableau. 

In [ ]:
df_salaries_filtered['cost_of_living_index'] = df_salaries_filtered['cost_of_living_index'].astype(str).str.replace('.', ',')
df_salaries_filtered['rent_index'] = df_salaries_filtered['rent_index'].astype(str).str.replace('.', ',')
df_salaries_filtered['cost_of_living_plus_rent_index'] = df_salaries_filtered['cost_of_living_plus_rent_index'].astype(str).str.replace('.', ',')
df_salaries_filtered['groceries_index'] = df_salaries_filtered['groceries_index'].astype(str).str.replace('.', ',')
df_salaries_filtered['restaurant_price_index'] = df_salaries_filtered['restaurant_price_index'].astype(str).str.replace('.', ',')
df_salaries_filtered['local_purchasing_power_index'] = df_salaries_filtered['local_purchasing_power_index'].astype(str).str.replace('.', ',')

In [ ]:
# Guardamos los dfs
df_salaries_filtered.to_csv('../data/clean/df_salaries_clean.csv', index=False)
df_pais_avg.to_csv('../data/clean/df_pais_avg.csv', index=False)